In [ ]:
from qTools import *

import Cahit as ck

import numpy as np

import matplotlib.pyplot as plt
import datetime

### System parameters

In [ ]:
resonatorDimension = 20
g = 1.79
qfreq = 0
resFreq = 2

### Simulation Parameters and Settings

In [ ]:
qSim = Simulation()
qSim.totalTime = 2
qSim.stepSize = 0.02
qSim.delStates = True

### Composite System Creation and Components

In [ ]:
JCSys = QuantumSystem(name='JC')

In [ ]:
cavJC = Cavity(dimension=resonatorDimension, frequency=resFreq)
qubJC = Qubit(frequency=qfreq)
JCSys.addSubSys([cavJC, qubJC])

In [ ]:
RabiSys = QuantumSystem(name='Rabi')

In [ ]:
#cavRabi = RabiSys.addSubSys(cavJC.copy())
cavRabi = RabiSys.addSubSys(Cavity(dimension=resonatorDimension, frequency=2*resFreq))
qubRabi = RabiSys.addSubSys(Qubit, frequency=qfreq)

In [ ]:
print(JCSys._freeEvol.simulation._stateBase__initialState._bound)
print(JCSys.simulation._stateBase__initialState)

#### Coupling term and initial state

In [ ]:
JCcoupling = JCSys.JC(g)

RabiCoupling = RabiSys.Rabi(g)

JCSys.initialState = [0,0]

RabiSys.initialState = [0,0]

# cavJC.initialState = 0
# qubJC.initialState = basis(2,0)

# cavRabi.initialState = 0
# qubRabi.initialState = basis(2,0)

### Define the Digital Algorithm

In [ ]:
JCunitary = freeEvolution(ratio=0.5, superSys=JCSys)
#JCunitary._createUnitary = 
bitFlip = xGate(implementation='instant', superSys=qubJC)

#AJC = freeEvolution(superSys=JCSys)
#qubset = AJC.createUpdate(system=qubJC, key='frequency', value=0)
AJCunitary = qProtocol(steps=[bitFlip, JCunitary, JCunitary, bitFlip])

digitalRabi = qProtocol(superSys=JCSys, steps=[JCunitary, AJCunitary, JCunitary], name='DigitalRabi')

### Add the systems into Simulation interface

In [ ]:
qSim.addSubSys(JCSys, digitalRabi)
qSim.addSubSys(RabiSys)

### Define Parameter Sweeps

In [ ]:
cavJCFreqSweep = qSim.Sweep.createSweep(system=cavJC, sweepKey='frequency', sweepList=np.arange(-2, 2+0.05, 0.05))

cavRabiFreqSweep = qSim.Sweep.createSweep(system=cavRabi, sweepKey='frequency', sweepList=np.arange(-4, 4+0.1, 0.1))

# stepSizeSweep = qSim.Sweep.createSweep(system=qSim, sweepKey='stepSize', sweepList=np.arange(0.001,0.1,0.001))
dimSweep1 = qSim.Sweep.createSweep(system=cavJC, sweepKey='dimension', sweepList=np.arange(2, 163, 1, dtype=np.int32))
dimSweep2 = qSim.Sweep.createSweep(system=cavRabi, sweepKey='dimension', sweepList=np.arange(2, 163, 1, dtype=np.int32))

### Define the (run-time) compute

In [ ]:
"""cavParity = parityEXP(cavJC.freeMat)
cavFree = cavJC.freeMat
subSysDimsOrdered = JCSys.subSysDimensions"""

def expectn(qSim, *args):
    JCs = list(qSim.subSys.values())[0]
    cav = JCs.getObjByName('Cavity1')
    cavParity = paritySUM(2*cav.dimension)
    cavFree = cav.freeMat
    subSysDimsOrdered = JCs.subSysDimensions
    # Observable expectations
    # Cavity Parity
    stateRabi = args[0]
    stateJC = args[1]

    exp = expectationKet(cavParity, stateRabi)
    qSim.getResultByName('RabiResults').result = ('Cavity Parity', exp)
    qSim.getResultByName('JCResults').result = ('Cavity Parity', expectationKet(cavParity, stateJC))
    # Photon number
    qSim.getResultByName('RabiResults').result = ('Photon Number', expectationKet(cavFree, stateRabi))
    qSim.getResultByName('JCResults').result = ('Photon Number', expectationKet(cavFree, stateJC))

    # Simulation fidelity
    qSim.qRes.result = ['simFidelity', fidelityKet(stateJC, stateRabi)]
    
    # Fidelity decay under resonator frequency perturbationN
    # if sweeping step size, it is required to use two different frequencies, so it is easier and more convenient to use 4 systems
    # protocols can be arranged to avoid extra exponentiation (in the digital case)
    # if doing multi-processing and sweeping resonator frequency, it is not possible to use the "previous time series" 
    # because they are simulated simultaneously on different cores
    # so the only option in both cases with multi-processing, it to use 4 systems
    #qSim.getResultByName('RabiResults').resultsKeyValList = ('FidelityDecay', fidelityKet(stateJC, stateJCPert))

    #qSim.getResultByName('JCDigitalRabiResults').resultsKeyValList = ('FidelityDecay', fidelityKet(stateRabi, stateRabiPert))

    # Fidelity to initial state
    # Below is another possibility to reach different results objects, all of above can be included below, which would simplfy the script a lot
    for protocol, qsys in qSim.subSys.items():
        # Fidelity to initial state
        qsys.qRes.result = ('Fidelity to initial', fidelityKet(protocol.currentState, qsys.initialState))

        # delocalisation measure (number of principle components)
        qsys.qRes.result = ('NPC', iprKetNB(protocol.currentState))

        # for the Wigner and qubit entropy, we need to take the partial trace. also, it is easier and more efficient to use partial traced state for Fock populations
        # Note: partial trace returns density matrices
        cavState = partialTrace(keep=[0], dims=subSysDimsOrdered, state=protocol.currentState)
        qubState = partialTrace(keep=[1], dims=subSysDimsOrdered, state=protocol.currentState)
        
        # Fock state populations by getting the diagonals.
        # the reason for np.real is that, even though it is real, it still has complex part that is zero, so this gets rid of extra information for the memory
        fockPops = np.real(cavState.diagonal()) # useful in two ways, obviously this and eigenvector statistics, so they should be turned into a method in QT
        #qsys.qRes.resultsKeyValList = ('Fock state populations', fockPops[0:10])
        qsys.qRes.resAverage= ('Fock state populations Ave', fockPops[0:10])
        del cavState
        # calculating Wigner and time-averaged Wigner
        #WignerFunc = Wigner(cavState, xvec=xvec)
        #protocol.qRes.resultsKeyValList = ('Wigner', WignerFunc)
        #protocol.qRes.averageKeyVal = ('Wigner average', WignerFunc)

        # calculating qubit entropy and its time average
        qubEntropy = entropy(qubState)
        del qubState
        qsys.qRes.result = ['Qubit entropy', qubEntropy]
        qsys.qRes.resAverage = ('Qubit entropy average', qubEntropy)

        # print(qsys.qRes.results)
        # print(qsys.qRes.states)

    # BELOW ARE THE TREE DIFFERENT POSSIBLE SYNTAX THAT IS ALREADY AVAILABLE

    #qSim.qRes.results['simFidelity'].append(fidelityKet(stateJC, stateRabi))
    #qSim.qRes.resultsMethod('simFidelity', fidelityKet(stateJC, stateRabi))
    #qSim.qRes.resultsKeyValList = ['simFidelity', fidelityKet(stateJC, stateRabi)]

    #qSim.getResultByName('RabiResults').results['cavParity'].append(expectationKet(cavParity, stateRabi))
    #qSim.getResultByName('RabiResults').resultsMethod('cavParity', expectationKet(cavParity, stateRabi))
    #qSim.getResultByName('RabiResults').resultsKeyValList = ('cavParity', expectationKet(cavParity, stateRabi))
    
    #qSim.getResultByName('JCDigitalRabiResults').results['cavParity'].append(expectationKet(cavParity, stateJC))
    #qSim.getResultByName('JCDigitalRabiResults').resultsMethod('cavParity', expectationKet(cavParity, stateJC))
    #qSim.getResultByName('JCDigitalRabiResults').resultsKeyValList = ('cavParity', expectationKet(cavParity, stateJC))

qSim.compute = expectn

### Run the simulation

In [ ]:
nw = datetime.datetime.now()

results = qSim.run(p=True, coreCount='all')

en = datetime.datetime.now()
print(en-nw)

In [ ]:
print(cavRabi.superSys)

In [ ]:
print(JCSys._freeEvol.numberOfExponentiations)

In [ ]:
print(qSim.qRes.results['simFidelity'])

In [ ]:
print(len(qSim.qRes.results['simFidelity']))

plt.plot([x*qSim.stepSize for x in range(qSim.stepCount)], qSim.qRes.results['simFidelity'], 'g-', label=r'Sim Fidelity ($\omega_r = 2$)')
plt.plot([x*qSim.stepSize for x in range(qSim.stepCount)], qSim.getResultByName('RabiResults').results['Cavity Parity'], 'b-.', label='Cav Parity Ideal')
plt.plot([x*qSim.stepSize for x in range(qSim.stepCount)], qSim.getResultByName('DigitalRabiResults').results['Cavity Parity'], 'r.', label='Cav Parity Digital')
plt.legend()
plt.xlabel(r'Time ($\mu s$)')

### Save Results

In [ ]:
path, fname = qSim.qRes.saveAll(fileName='trial', path='/Users/cahitkargi/Desktop', irregular=True)

### Plot the Results
---

Plotting is not part of the simulation library

In [ ]:
print(len(qSim.timeList))
print(len(qSim.qRes.results['simFidelity']), len(qSim.qRes.results['simFidelity'][0]))

In [ ]:
def plotRes(xList = cavJCFreqSweep.sweepList):
    fig = plt.figure(figsize=(12,9))
    setC = [0.94, 0.25, 0.02, 0.6]

    axGrids = ck.Plotting.Functions.grid(2, 3, fig=fig)

    ck.Plotting.SimplePlots.colorPlot(xList, qSim.timeList, qSim.qRes.results['simFidelity'], ax=axGrids[0], gif=fig, setC=setC)
    ck.Plotting.SimplePlots.colorPlot(xList, qSim.timeList, qSim.getResultByName('RabiResults').results['Cavity Parity'], ax=axGrids[1], gif=fig, setC=setC)
    ck.Plotting.SimplePlots.colorPlot(xList, qSim.timeList, qSim.getResultByName('JCResults').results['Cavity Parity'], ax=axGrids[2], gif=fig, setC=setC)

plotRes(cavJCFreqSweep.sweepList)

In [ ]:
def plotRes(xList = stepSizeSweep.sweepList):
    fig = plt.figure(figsize=(12,9))
    setC = [0.92, 0.25, 0.02, 0.6]

    axGrids = ck.Plotting.Functions.grid(8, 2, fig=fig)

    ck.Plotting.SimplePlots.colorPlotIreg(xList, qSim.finalTime, qSim.qRes.results['simFidelity'], ax=axGrids[14], gif=fig, setC=setC, mapC='GrYl', maxC=1, minC=0)

    ck.Plotting.SimplePlots.colorPlotIreg(xList, qSim.finalTime, qSim.getResultByName('RabiResults').results['Cavity Parity'], ax=axGrids[0], gif=fig, setC=setC)
    ck.Plotting.SimplePlots.colorPlotIreg(xList, qSim.finalTime, qSim.getResultByName('JCDigitalRabiResults').results['Cavity Parity'], ax=axGrids[1], gif=fig, setC=setC)


    ck.Plotting.SimplePlots.colorPlotIreg(xList, qSim.finalTime, qSim.getResultByName('RabiResults').results['Photon Number'], ax=axGrids[2], gif=fig, setC=setC, maxC=20, minC=0)
    ck.Plotting.SimplePlots.colorPlotIreg(xList, qSim.finalTime, qSim.getResultByName('JCDigitalRabiResults').results['Photon Number'], ax=axGrids[3], gif=fig, setC=setC, maxC=20, minC=0)


    ck.Plotting.SimplePlots.colorPlotIreg(xList, qSim.finalTime, qSim.getResultByName('RabiResults').results['Fidelity to initial'], ax=axGrids[4], gif=fig, setC=setC, maxC=1, minC=0)
    ck.Plotting.SimplePlots.colorPlotIreg(xList, qSim.finalTime, qSim.getResultByName('JCResults').results['Fidelity to initial'], ax=axGrids[5], gif=fig, setC=setC, maxC=1, minC=0)


    ck.Plotting.SimplePlots.colorPlotIreg(xList, qSim.finalTime, qSim.getResultByName('RabiResults').results['NPC'], ax=axGrids[6], gif=fig, setC=setC, maxC=20, minC=1)
    ck.Plotting.SimplePlots.colorPlotIreg(xList, qSim.finalTime, qSim.getResultByName('JCResults').results['NPC'], ax=axGrids[7], gif=fig, setC=setC, maxC=20, minC=1)


    ck.Plotting.SimplePlots.colorPlot(xList, np.arange(0, 10, 1), qSim.getResultByName('RabiResults').results['Fock state populations Ave'], ax=axGrids[12], gif=fig, setC=setC, maxC=0.4, minC=0)
    ck.Plotting.SimplePlots.colorPlot(xList, np.arange(0, 10, 1), qSim.getResultByName('JCResults').results['Fock state populations Ave'], ax=axGrids[13], gif=fig, setC=setC, maxC=0.4, minC=0)
    ck.Plotting.SimplePlots.colorPlotIreg(xList, qSim.finalTime, qSim.getResultByName('RabiResults').results['Qubit entropy'], ax=axGrids[8], gif=fig, setC=setC, maxC=1.7, minC=0)
    ck.Plotting.SimplePlots.colorPlotIreg(xList, qSim.finalTime, qSim.getResultByName('JCResults').results['Qubit entropy'], ax=axGrids[9], gif=fig, setC=setC, maxC=1.7, minC=0)
    axGrids[10].plot(xList, qSim.getResultByName('RabiResults').results['Qubit entropy average'])
    axGrids[11].plot(xList, qSim.getResultByName('JCResults').results['Qubit entropy average'])

plotRes()